In [ ]:
%pip install duckdb pandas jupysql matplotlib --quiet

## Building Boilerplate below:
Using DuckDB will allows me to explore my Docker's LocalStack Gold Layer created.

In [3]:
import os 
import duckdb 

from dotenv import load_dotenv

load_dotenv()

conn = duckdb.connect()
conn.execute("INSTALL httpfs; LOAD httpfs;")
conn.execute(f"""
    SET s3_endpoint='{os.getenv("LOCALSTACK_ENDPOINT").replace("http://", "")}';
    SET s3_use_ssl=false;
    SET s3_url_style='path';
    SET s3_access_key_id='{os.getenv("AWS_ACCESS_KEY_ID")}';
    SET s3_secret_access_key='{os.getenv("AWS_SECRET_ACCESS_KEY")}';
    SET s3_region='{os.getenv("AWS_DEFAULT_REGION_ID")}';
""")

Using SQL to write my Star Schema Query

In [6]:
query = """
SELECT 
    c.coin_name, 
    f.price_usd, 
    d.hour
FROM read_parquet('s3://crypto-raw-data/gold/fact_listings_*.parquet') f
JOIN read_parquet('s3://crypto-raw-data/gold/dim_coins_*.parquet') c ON f.coin_id = c.coin_id
JOIN read_parquet('s3://crypto-raw-data/gold/dim_date_*.parquet') d ON f.date_id = d.date_id
WHERE d.day = 6
ORDER BY f.price_usd DESC
LIMIT 100;
"""

df_results = conn.execute(query).df()
print(df_results)

                                 coin_name     price_usd  hour
0                       Maya Preferred PRA  6.475936e+08    13
1                       Maya Preferred PRA  6.475936e+08    13
2                       Maya Preferred PRA  6.475936e+08    13
3                              Jinbi Token  1.170139e+05    13
4                              Jinbi Token  1.170139e+05    13
..                                     ...           ...   ...
95  Blackrock, Inc. Tokenized Stock (Ondo)  1.125484e+03    13
96      Eli Lilly tokenized stock (xStock)  1.045843e+03    13
97      Eli Lilly tokenized stock (xStock)  1.045843e+03    13
98      Eli Lilly tokenized stock (xStock)  1.045843e+03    13
99        Eli Lilly Tokenized Stock (Ondo)  1.042922e+03    13

[100 rows x 3 columns]


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

